# Regression Evaluation

Notebook for testing regression using the California Housing dataset


# Setup


In [ ]:
import numpy as np
import pandas as pd
import scipy

# This is a draft---don't overengineer!
# NO renaming!
# NO refactoring!
# TODO: Remove this when the draft is done.

In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.metrics import mean_squared_error

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

## Config


In [ ]:
config = {
    "random_state": 15943,
    "test_size": 0.2,
    "n_splits": 5,
    "scoring": "neg_mean_squared_error",
}

# Data


In [ ]:
dataset = fetch_california_housing()

In [ ]:
X = pd.DataFrame(dataset["data"], columns=dataset["feature_names"])
X

In [ ]:
y = pd.DataFrame(dataset["target"], columns=dataset["target_names"])
y

In [ ]:
df = pd.concat([X, y], axis=1)

## EDA


In [ ]:
g = sns.PairGrid(df)
g.map_diag(sns.histplot, bins=32)
g.map_offdiag(sns.histplot, bins=32)

# Single-Feature Regression


## Set up


In [ ]:
results = {}

### Select data


In [ ]:
X_var = "MedInc"
y_var = "MedHouseVal"

In [ ]:
X = pd.DataFrame(X[X_var])
y = pd.DataFrame(y[y_var])

In [ ]:
fig = plt.figure()
ax = plt.gca()

ax.hist2d(
    X[X_var],
    y[y_var],
    bins=32,
)

ax.set_xlabel(X_var)
ax.set_ylabel(y_var)

### Split data


In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, KFold

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=config["test_size"], random_state=config["random_state"]
)

In [ ]:
cv = KFold(
    n_splits=config["n_splits"], shuffle=True, random_state=config["random_state"]
)

## Linear Regression


In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
# Make the estimator
model_name = "linear_regression"
model = LinearRegression()

In [ ]:
result = {}

In [ ]:
# Full prediction
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

In [ ]:
result["mse"] = mean_squared_error(y_test, y_pred)
result["mse"]

In [ ]:
# Crossval score
result["cross_val_score"] = cross_val_score(
    model, X_train, y_train, cv=cv, scoring=config["scoring"]
)
result["cross_val_score"]

In [ ]:
results[model_name] = result

## Compare Models


In [ ]:
# Format data
dfs = []
for key, value in results.items():

    df = pd.DataFrame(value)
    df["model_name"] = key
    dfs.append(df)
results_df = pd.concat(dfs)

In [ ]:
fig = plt.figure(figsize=(len(results) * 2, 2))
ax = plt.gca()

sns.swarmplot(
    data=results_df,
    x="model_name",
    y="cross_val_score",
)

ax.set_ylabel(config["scoring"])